# Tutorial Completo: Métodos Monte Carlo en Reinforcement Learning

## Objetivos de Aprendizaje

En este notebook aprenderás:
1. Los fundamentos matemáticos de los métodos Monte Carlo
2. La diferencia entre First-Visit y Every-Visit MC
3. MC Prediction para evaluación de políticas
4. MC Control On-Policy con ε-greedy
5. MC Control Off-Policy con Importance Sampling
6. Aplicaciones prácticas: Blackjack, GridWorld, Cliff Walking

**Duración estimada**: 3-4 horas

**Prerequisitos**: Haber completado el notebook de Dynamic Programming

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

# Configuración de visualización
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")
%matplotlib inline

# Importar nuestras implementaciones
sys.path.append("/home/user/Reinforcement-learning-guide")

from algoritmos_clasicos.monte_carlo.mc_prediction import (
    MCPrediction, SimpleGridWorld, SimpleBlackjack
)
from algoritmos_clasicos.monte_carlo.mc_control import (
    MCControlOnPolicy, MCControlOffPolicy, CliffWalking
)

print("✓ Importaciones exitosas")
print(f"NumPy version: {np.__version__}")